In [281]:
from random import random
import numpy as np
from itertools import combinations
import math
import random
import time as tm

In [282]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [283]:
# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [284]:
### create a graph with planted clique of size = clique_size

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    start_generate = tm.time()
    def plant_clique():
        for i in range(clique_size):
            for j in range(i + 1, clique_size):
                for l in range(j + 1, clique_size):
                    a = clique_vertices[i]
                    # print(a)
                    vec[a] += 1
                    b = clique_vertices[j]
                    # print(b)
                    vec[b] += 1
                    c = clique_vertices[l]
                    # print(c)
                    vec[c] += 1
                    add_all(a, b, c, g)

    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    # clique_vertices = random.sample(range(num_vertices), clique_size) # is is wrong??; would violate the clique size constraint
    
    clique_vertices = np.random.choice(range(num_vertices), clique_size, replace=False)

    plant_clique()

    return g, vec, clique_vertices


In [285]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [286]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == math.comb(active_count, 3):
        return True
    return False

In [287]:
# remove edge
def remove_edges(g, num_vertices, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph

In [288]:
# driver program

n = 100 # number of vertices
p = 0.5 # probablity of edge being included
k = 10 # clique size

# generate graph
start_generate = tm.time()
res = generate_graph(n, p, k)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertice in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0],  res[1], res[-1]

# print(f"graph: {G}\n")
G_0 = G.copy()
print(f"edge-occurence vector: {V}\n")
print(f"set of clique vertices after removal phase: {clique_vertices}\n")

time_generate = np.round(tm.time()-start_generate,3)
# print("Time taken to generate graph: ", "seconds")


edge-occurence vector: [2483 2363 2481 2438 2418 2422 2446 2428 2417 2408 2392 2399 2434 2402
 2388 2428 2484 2427 2429 2450 2437 2434 2367 2399 2483 2455 2431 2434
 2399 2353 2433 2436 2465 2461 2425 2487 2412 2477 2409 2441 2446 2434
 2450 2343 2409 2346 2412 2466 2352 2399 2420 2432 2438 2390 2336 2417
 2393 2394 2444 2362 2483 2417 2412 2370 2408 2320 2384 2344 2394 2473
 2487 2435 2374 2442 2392 2445 2381 2451 2388 2340 2372 2425 2452 2457
 2437 2437 2399 2456 2378 2394 2357 2474 2366 2436 2417 2416 2408 2422
 2462 2446]

set of clique vertices after removal phase: [33 12 39 32 83 84 70 69 97 87]



### Removal Phase

In [289]:
start_removal = tm.time()
iter = 0
removed = []

clique = is_clique(G, V, n)
while not clique:
    
    iter += 1

    curr = -1
    idx_sorted = np.argsort(V)
    
    print(f"i = {iter}\nindex sorted: {idx_sorted}")
    print(f"number of edges associated with each vertex: {V}")
    # print(f'current set of clique vertices: {clique_vertices}')

    for idx in range(n):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, n, V, curr)
    # print(A[1])
    G = A[0]

    clique = is_clique(G, V, n)
print("clique!!!")
time_remove = np.round(tm.time()-start_removal,3)

i = 1
index sorted: [65 54 79 43 67 45 48 29 90 59  1 92 22 63 80 72 88 76 66 14 78 53 10 74
 56 57 68 89 49 86 28 23 11 13 96 64  9 38 44 46 36 62 95 61 94 55  8  4
 50  5 97 34 81 17 15  7 18 26 51 30 12 21 41 27 71 93 31 20 84 85 52  3
 39 73 58 75 99 40  6 42 19 77 82 25 87 83 33 98 32 47 69 91 37  2  0 60
 24 16 70 35]
number of edges associated with each vertex: [2483 2363 2481 2438 2418 2422 2446 2428 2417 2408 2392 2399 2434 2402
 2388 2428 2484 2427 2429 2450 2437 2434 2367 2399 2483 2455 2431 2434
 2399 2353 2433 2436 2465 2461 2425 2487 2412 2477 2409 2441 2446 2434
 2450 2343 2409 2346 2412 2466 2352 2399 2420 2432 2438 2390 2336 2417
 2393 2394 2444 2362 2483 2417 2412 2370 2408 2320 2384 2344 2394 2473
 2487 2435 2374 2442 2392 2445 2381 2451 2388 2340 2372 2425 2452 2457
 2437 2437 2399 2456 2378 2394 2357 2474 2366 2436 2417 2416 2408 2422
 2462 2446]
vertex removed: 65 number of edges: 2320

i = 2
index sorted: [65 79 67 54 48 43 45 29 90 63  1 59 80 22 66 72 92 88 76 

In [290]:
iter

91

In [291]:
sorted(removed)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 85,
 86,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 98,
 99]

### Inclusion Phase

In [292]:
full_vertices = np.arange(n)
full_vertices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [293]:
included = np.setdiff1d(full_vertices, removed)
included

array([32, 33, 39, 69, 70, 83, 84, 87, 97])

In [294]:
def inclusion_phase(removed, in_set, G_0):
    
    def connected(idx, in_set):
        for j in range(len(in_set)):
                for k in range(j+1, len(in_set)):
                    if G_0[idx, in_set[j], in_set[k]] != 1:
                        print(f" vertex {idx} is not connected to all clique vertices")  
                        return False
        return True
                        
    for idx in removed:
        if connected(idx, in_set):
            print(f"add {idx} to clique!")
            in_set = np.append(in_set, idx)


    return in_set
    

In [295]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
time_include = np.round(tm.time()-start_include,3)

 vertex 65 is not connected to all clique vertices
 vertex 79 is not connected to all clique vertices
 vertex 67 is not connected to all clique vertices
 vertex 54 is not connected to all clique vertices
 vertex 48 is not connected to all clique vertices
 vertex 45 is not connected to all clique vertices
 vertex 29 is not connected to all clique vertices
 vertex 43 is not connected to all clique vertices
 vertex 1 is not connected to all clique vertices
 vertex 90 is not connected to all clique vertices
 vertex 66 is not connected to all clique vertices
 vertex 88 is not connected to all clique vertices
 vertex 80 is not connected to all clique vertices
 vertex 72 is not connected to all clique vertices
 vertex 63 is not connected to all clique vertices
 vertex 89 is not connected to all clique vertices
 vertex 76 is not connected to all clique vertices
 vertex 56 is not connected to all clique vertices
 vertex 53 is not connected to all clique vertices
add 12 to clique!
 vertex 59 is 

In [296]:
print(f"Set of clique vertices after generation phase: {clique_vertices}\n")
print(f"Set of clique vertices after removal phase: {included}, number of elements = {len(included)}\n")
print(f"set of vertices removed: {removed}, number of elements = {len(removed)}\n")
print(f"Set of clique vertices after inclusion phase: {res}")

Set of clique vertices after generation phase: [33 12 39 32 83 84 70 69 97 87]

Set of clique vertices after removal phase: [32 33 39 69 70 83 84 87 97], number of elements = 9

set of vertices removed: [65, 79, 67, 54, 48, 45, 29, 43, 1, 90, 66, 88, 80, 72, 63, 89, 76, 56, 53, 12, 59, 96, 36, 38, 8, 68, 22, 64, 55, 13, 71, 11, 27, 41, 10, 14, 75, 94, 5, 18, 30, 20, 58, 86, 34, 28, 15, 74, 21, 2, 42, 47, 91, 78, 46, 31, 3, 26, 6, 25, 85, 95, 92, 37, 0, 19, 9, 44, 51, 7, 99, 73, 52, 49, 4, 98, 24, 61, 50, 40, 60, 23, 93, 77, 35, 62, 81, 57, 16, 82, 17], number of elements = 91

Set of clique vertices after inclusion phase: [32 33 39 69 70 83 84 87 97 12]
